In [1]:
# main4.ipynb -> infinite loop

from classes.hero import Hero

from scenarios.basic_example import friendly_minions, enemy_minions
from scenarios.deck_example import paladin_deck, warrior_deck, add_card_to_hand

from state.transition_state import start_turn, apply_results, end_turn

from solver.solver import run_single_turn

In [2]:
def apply_results(solution, scenario_data, active_hero, opp_hero):
    """
    Example function to interpret the solver solution and apply damage.
    For a single turn, it updates hero HP if x_hero is used.
    You could also handle minion vs. minion damage or remove dead minions here.
    """
    if solution["status"] == 2:  # 2 => GRB.OPTIMAL
        print("Objective:", solution["objective"])

        # Access arrays from scenario_data
        A = scenario_data["A"]  # Attack values for friendly side
        # B = scenario_data["B"] # If you want to do something with friendly health
        # P = scenario_data["P"] # Enemy minion attacks
        # Q = scenario_data["Q"] # Enemy minion health
        # etc...

        # x_hero => which friendly minion attacked the opponent hero
        x_hero = solution.get("x_hero", {})
        total_hero_damage = 0
        for i, val in x_hero.items():
            if val > 0.5:
                dmg = A[i]
                print(f"Friendly minion {i} attacked {opp_hero.hero_class} hero for {dmg} damage.")
                total_hero_damage += dmg

        if total_hero_damage > 0:
            # If you track armor, you can reduce that first if you'd like:
            # e.g. if opp_hero.armor >= total_hero_damage:
            #       opp_hero.armor -= total_hero_damage
            #       total_hero_damage = 0
            #     else:
            #       total_hero_damage -= opp_hero.armor
            #       opp_hero.armor = 0
            opp_hero.health -= total_hero_damage
            print(f"{opp_hero.hero_class} hero's health is now {opp_hero.health}")

        # x_minions => minion i attacked minion j (if you want to show or handle that)
        if "x_minions" in solution:
            x_minions = solution["x_minions"]
            for (i, j), val in x_minions.items():
                if val > 0.5:
                    dmg = A[i]
                    print(f"Friendly minion {i} attacked enemy minion {j} for {dmg} damage.")
                    # If you want to track kills or remove minions, you'd do it here.

    else:
        print("No valid solution found. Status:", solution["status"])

def main():
    # 1) Create hero objects with your Hero class
    hero1 = Hero(hero_class="Paladin", health=15, armor=0)
    hero2 = Hero(hero_class="Warrior", health=15, armor=0)

    # 2) Each hero has an empty hand initially
    hero1_hand = []
    hero2_hand = []

    # 3) Let's have hero1 do a single turn, drawing from paladin_deck
    print(f"\n=== {hero1.hero_class}'s turn ===")
    start_turn(hero1, paladin_deck, hero1_hand)

    # 4) Build scenario data
    m = len(friendly_minions)        # on-board friendly minions
    n = len(enemy_minions)           # on-board enemy minions
    h = len(hero1_hand)             # cards in hero1's hand
    M = 5
    H_hero = hero2.health           # use hero2's current HP

    combined = friendly_minions + hero1_hand
    A = [mn.attack for mn in combined]
    B = [mn.health for mn in combined]
    P = [enm.attack for enm in enemy_minions]
    Q = [enm.health for enm in enemy_minions]
    C = [mn.mana_cost for mn in hero1_hand]
    S = [mn.strat_value for mn in hero1_hand]

    scenario_data = {
        "m": m, "n": n, "h": h,
        "M": M, "H_hero": H_hero,
        "A": A, "B": B,
        "P": P, "Q": Q,
        "C": C, "S": S
    }

    # 5) Run solver for this single turn
    solution = run_single_turn(scenario_data)

    # 6) Apply the results: interpret x_hero, x_minions, reduce HP, etc.
    apply_results(solution, scenario_data, hero1, hero2)

    # 7) End turn checks
    end_turn(hero1, hero2)

    print("\nTurn complete.")

if __name__ == "__main__":
    main()



=== Paladin's turn ===
Paladin draws: Faerie Dragon


TypeError: run_single_turn() missing 10 required positional arguments: 'n', 'h', 'M', 'H_hero', 'A', 'B', 'P', 'Q', 'C', and 'S'